In [1]:
!pip uninstall transformers torch torchvision -q -y 
!pip install transformers datasets torch torchvision -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv
/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv
/kaggle/input/nbme-score-clinical-patient-notes/features.csv
/kaggle/input/nbme-score-clinical-patient-notes/train.csv
/kaggle/input/nbme-score-clinical-patient-notes/test.csv


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from transformers import AutoTokenizer
from typing import List, Tuple

In [4]:
patient_notes = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv")
features = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/features.csv")
train = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/train.csv")

# merge dataset

train = pd.merge(train, patient_notes, on=["pn_num", "case_num"], how="left")
train = pd.merge(train, features, on=["feature_num", "case_num"], how="left")

In [5]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],HPI: 17yo M presents with palpitations. Patien...,Lightheaded


In [6]:
sample_text = train.iloc[0]["pn_history"]
sample_text

'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'

In [7]:
sample_text[:40]

'HPI: 17yo M presents with palpitations. '

In [8]:
def analyze_tokens_with_spaces(text: str, model_name: str, tokenizer_path: str) -> List[Tuple[Tuple[int, int], str]]:
    """Show exact token spans with spaces replaced from special markers."""
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    # Get encoding with offset mapping
    encoding = tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
    offsets = encoding['offset_mapping']
    
    # Create list of tuples with spans and processed token representation
    token_spans = []
    for token, (start, end) in zip(tokens, offsets):
        # Replace special markers with spaces
        if token.startswith('Ġ'):
            token = ' ' + token[1:]
        elif token.startswith('▁'):
            token = ' ' + token[1:]
        token_spans.append(((start, end), repr(token)))
    
    return token_spans

def print_token_analysis(text: str):
    """Print token analysis for each model."""
    models = {
        "BERT": "google-bert/bert-base-uncased",
        "RoBERTa": "FacebookAI/roberta-base",
        "DeBERTa": "microsoft/deberta-v3-base",
        "ModernBERT": "answerdotai/ModernBERT-base"
    }
    
    for model_name, model_path in models.items():
        token_spans = analyze_tokens_with_spaces(text, model_name, model_path)
        
        print(f"\n{model_name} token spans:")
        print("[")
        for (start, end), token in token_spans:
            # Ensure consistent spacing in output
            print(f"  [{start:>2}, {end:>2}), {token},")
        print("]")

In [9]:
text = "HPI: 17yo M presents with palpitations." #sample_text[:40]
print_token_analysis(text)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


BERT token spans:
[
  [ 0,  2), 'hp',
  [ 2,  3), '##i',
  [ 3,  4), ':',
  [ 5,  7), '17',
  [ 7,  9), '##yo',
  [10, 11), 'm',
  [12, 20), 'presents',
  [21, 25), 'with',
  [26, 29), 'pal',
  [29, 32), '##pit',
  [32, 38), '##ations',
  [38, 39), '.',
]


...


DeBERTa token spans:
[
  [ 0,  2), ' HP',
  [ 2,  3), 'I',
  [ 3,  4), ':',
  [ 4,  7), ' 17',
  [ 7,  9), 'yo',
  [ 9, 11), ' M',
  [11, 20), ' presents',
  [20, 25), ' with',
  [25, 38), ' palpitations',
  [38, 39), '.',
]


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]


ModernBERT token spans:
[
  [ 0,  1), 'H',
  [ 1,  3), 'PI',
  [ 3,  4), ':',
  [ 4,  7), ' 17',
  [ 7,  9), 'yo',
  [ 9, 11), ' M',
  [11, 20), ' presents',
  [20, 25), ' with',
  [25, 29), ' pal',
  [29, 32), 'pit',
  [32, 38), 'ations',
  [38, 39), '.',
]


So, Roberta and ModernBERT has similar tokenization approach 

Yet ModernBERT also consider whitespaces as Deberta 

i.e. [ 4,  7), ' 17' vs  [ 5,  7), ' 17',